# Notebook for hybrid decoding
Below are the pipelines used for the hybrid decoding. It first extracts the features for each pipeline individually, then, it concatenates these features and feeds this into an LDA. The feature extraction for c-VEP and alpha are directly from previous notebooks, however, for in order to get the P300 features in the correct format to concatenate it with the others, some adjustments had to be made.

### Imports

In [1]:
import os
import numpy as np
import joblib
import pyntbci
from scipy.signal import butter, sosfilt, hilbert
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.covariance import LedoitWolf
from mne.decoding import CSP
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

## Feature extraction
Below are the cells which are used to extract the features for c-VEP and alpha, respectively. The feature extraction for P300 was done in another notebook.

In [5]:
# -- c-VEP feature extraction --

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Extract c-VEP RCCA features per trial per subject and task.
@author: Adapted from Jordy Thielen
"""


# Paths
data_dir = '/Users/juliette/Desktop/thesis/preprocessing/c-VEP_preprocessing/c-VEP_ICA'
save_dir = '/Users/juliette/Desktop/thesis/features/c-VEP'
os.makedirs(save_dir, exist_ok=True)

# Subject and task lists
subjects = [
    "VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij", "VPpdik",
    "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit", "VPpdiu", "VPpdiv",
    "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"
]
tasks = ["overt", "covert"]

# RCCA settings
event = "dur"
onset_event = True
encoding_length = 0.3
ensemble = True

# Loop participants
for subject in subjects:
    print(f"{subject}", end="\t")

    for task in tasks:
        print(f"{task}: ", end="")

        # Load data
        fn = os.path.join(data_dir, f"sub-{subject}_task-{task}_ICA.npz")
        tmp = np.load(fn)
        fs = int(tmp["fs"])
        X = tmp["X"]
        y = tmp["y"]
        V = tmp["V"]

        # Fit RCCA model
        rcca = pyntbci.classifiers.rCCA(stimulus=V, fs=fs, event=event,
                                        encoding_length=encoding_length,
                                        onset_event=onset_event, ensemble=ensemble)
        rcca.fit(X, y)

        # Extract features (project trials into canonical space)
        features = rcca.decision_function(X)
        print(features.shape)
        features = features.reshape(features.shape[0], -1)  # shape: (n_trials, n_classes * n_components)

        save_path = os.path.join(save_dir, f"sub-{subject}_task-{task}_features.npz")
        np.savez(save_path, features=features, y=y)
        print(f"saved to {save_path}, shape: {features.shape}")


VPpdia	overt: (20, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdia_task-overt_features.npz, shape: (20, 2)
covert: (80, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdia_task-covert_features.npz, shape: (80, 2)
VPpdib	overt: (20, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdib_task-overt_features.npz, shape: (20, 2)
covert: (80, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdib_task-covert_features.npz, shape: (80, 2)
VPpdic	overt: (20, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdic_task-overt_features.npz, shape: (20, 2)
covert: (80, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdic_task-covert_features.npz, shape: (80, 2)
VPpdid	overt: (20, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdid_task-overt_features.npz, shape: (20, 2)
covert: (80, 2)
saved to /Users/juliette/Desktop/thesis/features/c-VEP/sub-VPpdid_task-covert_features.npz, shape: (80,

KeyboardInterrupt: 

In [2]:
# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    """
    Apply a bandpass filter to the data.
    """
    sos = butter(order, [lowcut, highcut], btype='band', fs=fs, output='sos')
    return sosfilt(sos, data, axis=-1)

# Hilbert transform feature function
def compute_average_hilbert_amplitude(data):
    """
    Compute log-mean amplitude using Hilbert transform.
    """
    analytic = hilbert(data, axis=2)
    amplitude = np.abs(analytic)
    mean_amplitude = amplitude.mean(axis=2)
    return np.log(mean_amplitude)

# Parameters
task = "covert"
n_comp = 4  # number of CSP components
subjects = [
    "VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij", "VPpdik",
    "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit", "VPpdiu", "VPpdiv",
    "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"
]
decoding_results_dir = '/Users/juliette/Desktop/thesis/features/alpha'

# Loop through subjects
for subject in subjects:
    print(f"Processing subject {subject}")
    
    # File paths
    file_dir = '/Users/juliette/Desktop/thesis/preprocessing/alpha_preprocessing/alpha_ICA'
    file_path = os.path.join(file_dir, f"sub-{subject}_task-{task}_alpha_ICA.npz")

    # Check file exists
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    # Load data
    npz_data = np.load(file_path)
    X = npz_data['X']  # EEG data: trials x channels x samples
    y = npz_data['y']  # Labels: trials
    fs = npz_data['fs'].flatten()[0]  # Sampling frequency as integer

    print("Shape of X:", X.shape)

    # Preprocessing
    X = bandpass_filter(X, 8, 12, fs=fs)  # Alpha band filtering
    X = X[:, :, 120:-120]  # Remove edge artifacts

    # CSP and feature extraction
    csp = CSP(n_components=n_comp, reg=0.01, log=None, transform_into='csp_space')
    X_csp = csp.fit_transform(X, y)  # Apply CSP
    features = compute_average_hilbert_amplitude(X_csp)  # Extract features

    print(f"Extracted features shape for subject {subject}: {features.shape}")

    # Save features
    if not os.path.exists(decoding_results_dir):
        os.makedirs(decoding_results_dir)

    out_path = os.path.join(decoding_results_dir, f"sub-{subject}_task-{task}_alpha_features.npz")
    np.savez(out_path, features=features, labels=y)

    print(f"Saved features for subject {subject} to {out_path}")


Processing subject VPpdia
Shape of X: (80, 62, 10239)
Computing rank from data with rank=None
    Using tolerance 0.00013 (2.2e-16 eps * 62 dim * 9.3e+09  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 0.00013 (2.2e-16 eps * 62 dim * 9.5e+09  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using SHRINKAGE
Done.
Extracted features shape for subject VPpdia: (80, 4)
Saved features for subject VPpdia to /Users/juliette/Desktop/thesis/features/alpha/sub-VPpdia_task-covert_alpha_features.npz
Processing subject VPpdib
Shape of X: (80, 61, 10239)
Computing rank from data with rank=None
    Using tolerance 0.00051 (2.2e-16 eps * 61 dim * 3.8e+10  max singular value)
    Est

Shape of X: (80, 63, 10239)
Computing rank from data with rank=None
    Using tolerance 0.00013 (2.2e-16 eps * 63 dim * 9.4e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 0.00013 (2.2e-16 eps * 63 dim * 9.3e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using SHRINKAGE
Done.
Extracted features shape for subject VPpdik: (80, 4)
Saved features for subject VPpdik to /Users/juliette/Desktop/thesis/features/alpha/sub-VPpdik_task-covert_alpha_features.npz
Processing subject VPpdil
Shape of X: (80, 63, 10239)
Computing rank from data with rank=None
    Using tolerance 0.00016 (2.2e-16 eps * 63 dim * 1.1e+10  max singular value)
    Estimated rank (mag): 63
    

Computing rank from data with rank=None
    Using tolerance 0.00011 (2.2e-16 eps * 63 dim * 7.9e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 0.00011 (2.2e-16 eps * 63 dim * 8.1e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using SHRINKAGE
Done.
Extracted features shape for subject VPpdiu: (80, 4)
Saved features for subject VPpdiu to /Users/juliette/Desktop/thesis/features/alpha/sub-VPpdiu_task-covert_alpha_features.npz
Processing subject VPpdiv
Shape of X: (80, 60, 10239)
Computing rank from data with rank=None
    Using tolerance 0.00034 (2.2e-16 eps * 60 dim * 2.6e+10  max singular value)
    Estimated rank (mag): 60
    MAG: rank 60 computed from 6

## Helper functions
Below are the helper functions used for the P300 decoding.

In [2]:
def balance_classes(X, y, ratio_0_to_1=1.0):
    
    """
    Sub-select X and y based on a specified ratio of 0s to 1s, keeping the original order.

    Parameters:
    X (numpy.ndarray): Feature matrix of shape (n_samples, n_features).
    y (numpy.ndarray): Label vector of shape (n_samples,).
    ratio_0_to_1 (float): The desired ratio of 0s to 1s in the balanced dataset.

    Returns:
    X_balanced, y_balanced: Sub-selected feature matrix and label vector.
    """
    # Step 1: Identify indices of 0s and 1s
    indices_0 = np.where(y == 0)[0]
    indices_1 = np.where(y == 1)[0]
    
    # Step 2: Calculate the number of samples to select for each class
    num_1s = len(indices_1)
    num_0s = min(len(indices_0), int(num_1s * ratio_0_to_1))
    
    # Step 3: Randomly sample the desired number of 0s and 1s
    selected_indices_0 = np.random.choice(indices_0, num_0s, replace=False)
    selected_indices_1 = np.random.choice(indices_1, num_1s, replace=False)
    
    # Step 4: Combine selected indices and sort to preserve original order
    balanced_indices = np.sort(np.concatenate([selected_indices_0, selected_indices_1]))
    
    # Step 5: Sub-select X and y based on the balanced indices
    X_balanced = X[balanced_indices]
    y_balanced = y[balanced_indices]
    
    return X_balanced, y_balanced

def filter_valid_epochs(X, y, z=None, return_mask=False):
    """
    Filters out epochs where either the features in X or the labels in y contain NaN values.
    Optionally, if a z array is provided, it is filtered similarly.
    
    Parameters:
        X (np.ndarray): A 2D numpy array with shape (n_epochs, n_features).
        y (np.ndarray): A 1D numpy array with shape (n_epochs,).
        z (np.ndarray, optional): An array that will be filtered using the same mask.
        return_mask (bool, optional): If True, the boolean mask used for filtering is returned.
    
    Returns:
        filtered_X (np.ndarray): X with only rows that have no NaN values.
        filtered_y (np.ndarray): y with only entries corresponding to valid epochs.
        filtered_z (np.ndarray or None): Filtered z array (if provided) or None.
        mask (np.ndarray, optional): The boolean mask of valid epochs; only returned if return_mask=True.
    """
    # Create a mask for valid labels and features
    valid_label_mask = ~np.isnan(y)
    valid_feature_mask = ~np.isnan(X).any(axis=1)
    combined_mask = valid_label_mask & valid_feature_mask

    # Apply the mask to X and y
    filtered_X = X[combined_mask]
    filtered_y = y[combined_mask]
    
    if z is not None:
        filtered_z = z[combined_mask]
    else:
        filtered_z = None

    if return_mask:
        return filtered_X, filtered_y, filtered_z, combined_mask
    else:
        return filtered_X, filtered_y, filtered_z

# Hybrid decoding
The cell below contains the decoding for P300, alpha and c-VEP together.

In [2]:
# Paths
cvep_features_dir = '/Users/juliette/Desktop/thesis/features/c-VEP'  # where c-VEP features are stored
alpha_features_dir = '/Users/juliette/Desktop/thesis/features/alpha'  # where alpha features are stored
p300_features_dir = '/Users/juliette/Desktop/thesis/preprocessing/features/with_ICA'  # where P300 features are stored
save_dir = '/Users/juliette/Desktop/thesis/results/hybrid_simple/alpha+p300+c-vep'

# Subject and task lists
subjects = [
    "VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij", "VPpdik",
    "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit", "VPpdiu", "VPpdiv",
    "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"
]
tasks = ["covert"]
subject_accuracies = []
subject_std = []

# Loop over subjects and tasks
for subject in subjects:
    for task in tasks:
        print(f"Loading features for {subject}, task={task}")

        # Load c-VEP features
        cvep_file_path = os.path.join(cvep_features_dir, f"sub-{subject}_task-{task}_features.npz")
        if os.path.exists(cvep_file_path):
            cvep_data = np.load(cvep_file_path)
            X_cvep = cvep_data['features']  # Features for c-VEP task (trials x features)
            print(X_cvep.shape)
            y_cvep = cvep_data['y']  # Labels for c-VEP task
            print(f"Loaded c-VEP features: {X_cvep.shape}")
        else:
            print(f"Warning: c-VEP features file not found for {subject}, task={task}")

        # Load alpha features
        alpha_file_path = os.path.join(alpha_features_dir, f"sub-{subject}_task-{task}_alpha_features.npz")
        if os.path.exists(alpha_file_path):
            alpha_data = np.load(alpha_file_path)
            X_alpha = alpha_data['features']  # Features for alpha task (trials x features)
            y_alpha = alpha_data['labels']  # Labels for alpha task
            print(f"Loaded alpha features: {X_alpha.shape}")
        else:
            print(f"Warning: alpha features file not found for {subject}, task={task}")

        # Load P300 features
        file_path_p300 = os.path.join(p300_features_dir, f"sub-{subject}", f"sub-{subject}_task-{task}_p300_features_ICA.npz")
        if os.path.exists(file_path_p300):
            p300_features = np.load(file_path_p300)

            X_p300 = p300_features['X']  # Shape: trials x epochs x channels x features
            y_p300 = p300_features['y']  # Labels indicating cued side: trials
            z_p300 = p300_features['z']  # Left and right targets: trials x epochs x sides
            fs_p300 = p300_features['fs']
            print(f"Loaded p300 features: {X_p300.shape}")

        # Flatten z_p300 to (trials * epochs, sides)
        z_p300_flat = z_p300.reshape(-1, z_p300.shape[2])  # Shape: (trials * epochs, sides)

        # Find the number of epochs per trial
        epochs_per_trial = X_p300.shape[1]  # Number of epochs per trial
        print("X_p300.shape:", X_p300.shape)

        # Initialize lists to store averaged epochs for each trial
        left_target_averaged_trials = []
        right_target_averaged_trials = []

        # Loop over each trial
        for i_trial in range(X_p300.shape[0]):
            # Extract the epochs for the current trial (shape: epochs x channels x features)
            trial_epochs = X_p300[i_trial]

            # Extract the target labels for the current trial (shape: epochs x 2)
            trial_targets = z_p300_flat[i_trial * epochs_per_trial: (i_trial + 1) * epochs_per_trial]

            # Average epochs for left target (assuming 1 = left target)
            left_target_epochs = trial_epochs[trial_targets[:, 0] == 1]
            if len(left_target_epochs) > 0:
                left_target_averaged = np.mean(left_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                left_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no left target, set to zeros

            # Average epochs for right target (assuming 1 = right target)
            right_target_epochs = trial_epochs[trial_targets[:, 1] == 1]
            if len(right_target_epochs) > 0:
                right_target_averaged = np.mean(right_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                right_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no right target, set to zeros

            # Store the averaged epochs for the current trial
            left_target_averaged_trials.append(left_target_averaged)
            right_target_averaged_trials.append(right_target_averaged)

        # Convert lists to numpy arrays
        left_target_averaged_trials = np.array(left_target_averaged_trials)
        print("Shape of left_target_averaged_trials:", left_target_averaged_trials.shape)
        right_target_averaged_trials = np.array(right_target_averaged_trials)

        # Concatenate the averaged features from both groups (left and right)
        X_p300_averaged = np.concatenate([left_target_averaged_trials, right_target_averaged_trials], axis=1)

        # Flatten the last two dimensions (channels and features)
        X_p300_averaged_flat = X_p300_averaged.reshape(X_p300_averaged.shape[0], -1)
        # Concatenate the features of the P300, alpha and c-VEP
        X_combined = np.concatenate([X_p300_averaged_flat, X_alpha, X_cvep], axis=1)
        print("shape of X_combined:", X_combined.shape)

        # Cross-validation setup
        fold_accuracies = []
        fold_roc_auc = []
        n_folds = 4
        n_trials = X_combined.shape[0] // n_folds
        folds = np.repeat(np.arange(n_folds), n_trials)

        for i_fold in range(n_folds):
            print(f"  Fold {i_fold + 1}/{n_folds}")

            # Split train and test data
            X_trn, y_trn = X_combined[folds != i_fold, :], y_cvep[folds != i_fold]
            X_tst, y_tst = X_combined[folds == i_fold, :], y_cvep[folds == i_fold]

            # LDA classifier
            lda = LinearDiscriminantAnalysis(solver="lsqr", covariance_estimator=LedoitWolf())
            lda.fit(X_trn, y_trn)

            # Make predictions
            y_pred = lda.predict(X_tst)

            # Compute performance metrics
            accuracy = accuracy_score(y_tst, y_pred)
            print(f"Fold {i_fold+1} accuracy: {accuracy}")

            fold_accuracies.append(accuracy)

        # Calculate average accuracy over all folds
        average_accuracy = np.mean(fold_accuracies)
        subject_accuracies.append(fold_accuracies)
        subject_std.append(np.std(fold_accuracies))  # compute std over folds
        print(f"Average accuracy over all folds: {average_accuracy}\n STD: {np.std(fold_accuracies):.2f}")

# grand_average_accuracy = np.mean(subject_accuracies)
# print(f"\nGrand average accuracy across all subjects: {grand_average_accuracy:.4f}")

# # Convert list to array
# subject_accuracies_array = np.array(subject_accuracies)
# subject_std_array = np.array(subject_std)

# # Save the results
# save_path = os.path.join(save_dir, 'cvep_p300_alpha_hybrid_accuracy_results.npz')
# np.savez(save_path,
#          accuracy=subject_accuracies_array,
#          std=subject_std_array,
#          subjects=subjects,
#          tasks=tasks,
#          n_folds=n_folds,
#          method='hybrid')

# print(f"\nSaved results to: {save_path}")

Loading features for VPpdia, task=covert
(80, 2)
Loaded c-VEP features: (80, 2)
Loaded alpha features: (80, 4)
Loaded p300 features: (80, 80, 62, 6)
X_p300.shape: (80, 80, 62, 6)
Shape of left_target_averaged_trials: (80, 62, 6)
shape of X_combined: (80, 750)
  Fold 1/4
Fold 1 accuracy: 0.8
  Fold 2/4
Fold 2 accuracy: 0.75
  Fold 3/4
Fold 3 accuracy: 0.55
  Fold 4/4
Fold 4 accuracy: 0.8
Average accuracy over all folds: 0.7250000000000001
 STD: 0.10
Loading features for VPpdib, task=covert
(80, 2)
Loaded c-VEP features: (80, 2)
Loaded alpha features: (80, 4)
Loaded p300 features: (80, 80, 61, 6)
X_p300.shape: (80, 80, 61, 6)
Shape of left_target_averaged_trials: (80, 61, 6)
shape of X_combined: (80, 738)
  Fold 1/4
Fold 1 accuracy: 1.0
  Fold 2/4


KeyboardInterrupt: 

# Pairwise decoding
Now the pairwise combinations are presented.

In [4]:
# -- c-VEP AND ALPHA

# Paths
cvep_features_dir = '/Users/juliette/Desktop/thesis/features/c-VEP'  # where c-VEP features are stored
alpha_features_dir = '/Users/juliette/Desktop/thesis/features/alpha'  # where alpha features are stored
p300_features_dir = '/Users/juliette/Desktop/thesis/preprocessing/features/with_ICA'  # where P300 features are stored
save_dir = '/Users/juliette/Desktop/thesis/results/hybrid_simple/alpha+c-vep'


# Subject and task lists
subjects = [
    "VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij", "VPpdik",
    "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit", "VPpdiu", "VPpdiv",
    "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"
]
tasks = ["covert"]
subject_accuracies = []
subject_std = []


# Loop over subjects and tasks
for subject in subjects:
    for task in tasks:
        print(f"Loading features for {subject}, task={task}")

        # Load c-VEP features
        cvep_file_path = os.path.join(cvep_features_dir, f"sub-{subject}_task-{task}_features.npz")
        if os.path.exists(cvep_file_path):
            cvep_data = np.load(cvep_file_path)
            X_cvep = cvep_data['features']  # Features for c-VEP task (trials x features)
            y_cvep = cvep_data['y']  # Labels for c-VEP task
            print(f"Loaded c-VEP features: {X_cvep.shape}")
        else:
            print(f"Warning: c-VEP features file not found for {subject}, task={task}")

        # Load alpha features
        alpha_file_path = os.path.join(alpha_features_dir, f"sub-{subject}_task-{task}_alpha_features.npz")
        if os.path.exists(alpha_file_path):
            alpha_data = np.load(alpha_file_path)
            X_alpha = alpha_data['features']  # Features for alpha task (trials x features)
            y_alpha = alpha_data['labels']  # Labels for alpha task
            print(f"Loaded alpha features: {X_alpha.shape}")
        else:
            print(f"Warning: alpha features file not found for {subject}, task={task}")

        # Concatenate the features of the P300, alpha and c-VEP
        X_combined = np.concatenate([X_alpha, X_cvep], axis=1)

        # Cross-validation setup
        fold_accuracies = []
        fold_roc_auc = []
        n_folds = 4
        n_trials = X_combined.shape[0] // n_folds
        folds = np.repeat(np.arange(n_folds), n_trials)

        for i_fold in range(n_folds):
            print(f"  Fold {i_fold + 1}/{n_folds}")

            # Split train and test data
            X_trn, y_trn = X_combined[folds != i_fold, :], y_cvep[folds != i_fold]
            X_tst, y_tst = X_combined[folds == i_fold, :], y_cvep[folds == i_fold]

            # LDA classifier
            lda = LinearDiscriminantAnalysis(solver="lsqr", covariance_estimator=LedoitWolf())
            lda.fit(X_trn, y_trn)

            # Make predictions
            y_pred = lda.predict(X_tst)

            # Compute performance metrics
            accuracy = accuracy_score(y_tst, y_pred)
            print(f"Fold {i_fold+1} accuracy: {accuracy}")

            fold_accuracies.append(accuracy)

        # Calculate average accuracy over all folds
        average_accuracy = np.mean(fold_accuracies)
        subject_accuracies.append(fold_accuracies)
        subject_std.append(np.std(fold_accuracies))  # compute std over folds
        print(f"Average accuracy over all folds: {average_accuracy}\n STD: {np.std(fold_accuracies):.2f}")

grand_average_accuracy = np.mean(subject_accuracies)
print(f"\nGrand average accuracy across all subjects: {grand_average_accuracy:.4f}")

# Convert list to array
subject_accuracies_array = np.array(subject_accuracies)
subject_std_array = np.array(subject_std)

# Save the results
save_path = os.path.join(save_dir, 'cvep_alpha_hybrid_accuracy_results.npz')
np.savez(save_path,
         accuracy=subject_accuracies_array,
         std=subject_std_array,
         subjects=subjects,
         tasks=tasks,
         n_folds=n_folds,
         method='hybrid')

print(f"\nSaved results to: {save_path}")

Loading features for VPpdia, task=covert
Loaded c-VEP features: (80, 2)
Loaded alpha features: (80, 4)
  Fold 1/4
Fold 1 accuracy: 0.6
  Fold 2/4
Fold 2 accuracy: 0.75
  Fold 3/4
Fold 3 accuracy: 0.55
  Fold 4/4
Fold 4 accuracy: 0.5
Average accuracy over all folds: 0.6000000000000001
 STD: 0.09
Loading features for VPpdib, task=covert
Loaded c-VEP features: (80, 2)
Loaded alpha features: (80, 4)
  Fold 1/4
Fold 1 accuracy: 1.0
  Fold 2/4
Fold 2 accuracy: 1.0
  Fold 3/4
Fold 3 accuracy: 1.0
  Fold 4/4
Fold 4 accuracy: 1.0
Average accuracy over all folds: 1.0
 STD: 0.00
Loading features for VPpdic, task=covert
Loaded c-VEP features: (80, 2)
Loaded alpha features: (80, 4)
  Fold 1/4
Fold 1 accuracy: 1.0
  Fold 2/4
Fold 2 accuracy: 1.0
  Fold 3/4
Fold 3 accuracy: 0.85
  Fold 4/4
Fold 4 accuracy: 1.0
Average accuracy over all folds: 0.9625
 STD: 0.06
Loading features for VPpdid, task=covert
Loaded c-VEP features: (80, 2)
Loaded alpha features: (80, 4)
  Fold 1/4
Fold 1 accuracy: 0.9
  Fold 

In [5]:
print("Flattened (2*channels, features):", X.reshape(2 * ch, f).flatten())
print("Flattened (channels, 2*features):", X.reshape(ch, 2 * f).flatten())


NameError: name 'ch' is not defined

In [6]:
# -- ALPHA AND P300

# Paths
cvep_features_dir = '/Users/juliette/Desktop/thesis/features/c-VEP'  # where c-VEP features are stored
alpha_features_dir = '/Users/juliette/Desktop/thesis/features/alpha'  # where alpha features are stored
p300_features_dir = '/Users/juliette/Desktop/thesis/preprocessing/features/with_ICA'  # where P300 features are stored
save_dir = '/Users/juliette/Desktop/thesis/results/hybrid_simple/alpha+p300'


# Subject and task lists
subjects = [
    "VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij", "VPpdik",
    "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit", "VPpdiu", "VPpdiv",
    "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"
]
tasks = ["covert"]
subject_accuracies = []
subject_std = []

# Loop over subjects and tasks
for subject in subjects:
    for task in tasks:
        print(f"Loading features for {subject}, task={task}")

        # Load alpha features
        alpha_file_path = os.path.join(alpha_features_dir, f"sub-{subject}_task-{task}_alpha_features.npz")
        if os.path.exists(alpha_file_path):
            alpha_data = np.load(alpha_file_path)
            X_alpha = alpha_data['features']  # Features for alpha task (trials x features)
            y_alpha = alpha_data['labels']  # Labels for alpha task
            print(f"Loaded alpha features: {X_alpha.shape}")
        else:
            print(f"Warning: alpha features file not found for {subject}, task={task}")

        # Load P300 features
        file_path_p300 = os.path.join(p300_features_dir, f"sub-{subject}", f"sub-{subject}_task-{task}_p300_features_ICA.npz")
        if os.path.exists(file_path_p300):
            p300_features = np.load(file_path_p300)

            X_p300 = p300_features['X']  # Shape: trials x epochs x channels x features
            y_p300 = p300_features['y']  # Labels indicating cued side: trials
            z_p300 = p300_features['z']  # Left and right targets: trials x epochs x sides
            fs_p300 = p300_features['fs']
            print(f"Loaded p300 features: {X_p300.shape}")

        # Flatten z_p300 to (trials * epochs, sides)
        z_p300_flat = z_p300.reshape(-1, z_p300.shape[2])  # Shape: (trials * epochs, sides)

        # Find the number of epochs per trial
        epochs_per_trial = X_p300.shape[1]  # Number of epochs per trial

        # Initialize lists to store averaged epochs for each trial
        left_target_averaged_trials = []
        right_target_averaged_trials = []

        # Loop over each trial
        for i_trial in range(X_p300.shape[0]):
            # Extract the epochs for the current trial (shape: epochs x channels x features)
            trial_epochs = X_p300[i_trial]

            # Extract the target labels for the current trial (shape: epochs x 2)
            trial_targets = z_p300_flat[i_trial * epochs_per_trial: (i_trial + 1) * epochs_per_trial]

            # Average epochs for left target (assuming 1 = left target)
            left_target_epochs = trial_epochs[trial_targets[:, 0] == 1]
            if len(left_target_epochs) > 0:
                left_target_averaged = np.mean(left_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                left_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no left target, set to zeros

            # Average epochs for right target (assuming 1 = right target)
            right_target_epochs = trial_epochs[trial_targets[:, 1] == 1]
            if len(right_target_epochs) > 0:
                right_target_averaged = np.mean(right_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                right_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no right target, set to zeros

            # Store the averaged epochs for the current trial
            left_target_averaged_trials.append(left_target_averaged)
            right_target_averaged_trials.append(right_target_averaged)

        # Convert lists to numpy arrays
        left_target_averaged_trials = np.array(left_target_averaged_trials)
        right_target_averaged_trials = np.array(right_target_averaged_trials)

        # Concatenate the averaged features from both groups (left and right)
        X_p300_averaged = np.concatenate([left_target_averaged_trials, right_target_averaged_trials], axis=1)

        # Flatten the last two dimensions (channels and features)
        X_p300_averaged_flat = X_p300_averaged.reshape(X_p300_averaged.shape[0], -1)
        
        # Concatenate the features of the P300, alpha and c-VEP
        X_combined = np.concatenate([X_p300_averaged_flat, X_alpha], axis=1)

        # Cross-validation setup
        fold_accuracies = []
        fold_roc_auc = []
        n_folds = 4
        n_trials = X_combined.shape[0] // n_folds
        folds = np.repeat(np.arange(n_folds), n_trials)

        for i_fold in range(n_folds):
            print(f"  Fold {i_fold + 1}/{n_folds}")

            # Split train and test data
            X_trn, y_trn = X_combined[folds != i_fold, :], y_p300[folds != i_fold]
            X_tst, y_tst = X_combined[folds == i_fold, :], y_p300[folds == i_fold]

            # LDA classifier
            lda = LinearDiscriminantAnalysis(solver="lsqr", covariance_estimator=LedoitWolf())
            lda.fit(X_trn, y_trn)

            # Make predictions
            y_pred = lda.predict(X_tst)

            # Compute performance metrics
            accuracy = accuracy_score(y_tst, y_pred)
            print(f"Fold {i_fold+1} accuracy: {accuracy}")

            fold_accuracies.append(accuracy)

        # Calculate average accuracy over all folds
        average_accuracy = np.mean(fold_accuracies)
        subject_accuracies.append(fold_accuracies)
        subject_std.append(np.std(fold_accuracies))  # compute std over folds
        print(f"Average accuracy over all folds: {average_accuracy}\n STD: {np.std(fold_accuracies):.2f}")

grand_average_accuracy = np.mean(subject_accuracies)
print(f"\nGrand average accuracy across all subjects: {grand_average_accuracy:.4f}")

# Convert list to array
subject_accuracies_array = np.array(subject_accuracies)
subject_std_array = np.array(subject_std)

# Save the results
save_path = os.path.join(save_dir, 'p300_alpha_hybrid_accuracy_results.npz')
np.savez(save_path,
         accuracy=subject_accuracies_array,
         std=subject_std_array,
         subjects=subjects,
         tasks=tasks,
         n_folds=n_folds,
         method='hybrid')

print(f"\nSaved results to: {save_path}")


Loading features for VPpdia, task=covert
Loaded alpha features: (80, 4)
Loaded p300 features: (80, 80, 62, 6)
  Fold 1/4
Fold 1 accuracy: 0.4
  Fold 2/4
Fold 2 accuracy: 0.6
  Fold 3/4
Fold 3 accuracy: 0.65
  Fold 4/4
Fold 4 accuracy: 0.4
Average accuracy over all folds: 0.5125
 STD: 0.11
Loading features for VPpdib, task=covert
Loaded alpha features: (80, 4)
Loaded p300 features: (80, 80, 61, 6)
  Fold 1/4
Fold 1 accuracy: 1.0
  Fold 2/4
Fold 2 accuracy: 1.0
  Fold 3/4
Fold 3 accuracy: 1.0
  Fold 4/4
Fold 4 accuracy: 1.0
Average accuracy over all folds: 1.0
 STD: 0.00
Loading features for VPpdic, task=covert
Loaded alpha features: (80, 4)
Loaded p300 features: (80, 80, 63, 6)
  Fold 1/4
Fold 1 accuracy: 1.0
  Fold 2/4
Fold 2 accuracy: 1.0
  Fold 3/4
Fold 3 accuracy: 0.85
  Fold 4/4
Fold 4 accuracy: 1.0
Average accuracy over all folds: 0.9625
 STD: 0.06
Loading features for VPpdid, task=covert
Loaded alpha features: (80, 4)
Loaded p300 features: (80, 80, 63, 6)
  Fold 1/4
Fold 1 accura

Fold 1 accuracy: 0.6
  Fold 2/4
Fold 2 accuracy: 0.6
  Fold 3/4
Fold 3 accuracy: 0.7
  Fold 4/4
Fold 4 accuracy: 0.65
Average accuracy over all folds: 0.6375
 STD: 0.04

Grand average accuracy across all subjects: 0.9116

Saved results to: /Users/juliette/Desktop/thesis/results/hybrid_simple/alpha+p300/p300_alpha_hybrid_accuracy_results.npz


In [5]:
# -- c-VEP AND P300

# Paths
cvep_features_dir = '/Users/juliette/Desktop/thesis/features/c-VEP'  # where c-VEP features are stored
alpha_features_dir = '/Users/juliette/Desktop/thesis/features/alpha'  # where alpha features are stored
p300_features_dir = '/Users/juliette/Desktop/thesis/preprocessing/features/with_ICA'  # where P300 features are stored
save_dir = '/Users/juliette/Desktop/thesis/results/hybrid_simple/p300+cvep'
subject_std = []

# Subject and task lists
subjects = [
    "VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij", "VPpdik",
    "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit", "VPpdiu", "VPpdiv",
    "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"
]
tasks = ["covert"]
subject_accuracies = []

# Loop over subjects and tasks
for subject in subjects:
    for task in tasks:
        print(f"Loading features for {subject}, task={task}")

        # Load c-VEP features
        cvep_file_path = os.path.join(cvep_features_dir, f"sub-{subject}_task-{task}_features.npz")
        if os.path.exists(cvep_file_path):
            cvep_data = np.load(cvep_file_path)
            X_cvep = cvep_data['features']  # Features for c-VEP task (trials x features)
            y_cvep = cvep_data['y']  # Labels for c-VEP task
            print(f"Loaded c-VEP features: {X_cvep.shape}")
        else:
            print(f"Warning: c-VEP features file not found for {subject}, task={task}")

        # Load P300 features
        file_path_p300 = os.path.join(p300_features_dir, f"sub-{subject}", f"sub-{subject}_task-{task}_p300_features_ICA.npz")
        if os.path.exists(file_path_p300):
            p300_features = np.load(file_path_p300)

            X_p300 = p300_features['X']  # Shape: trials x epochs x channels x features
            y_p300 = p300_features['y']  # Labels indicating cued side: trials
            z_p300 = p300_features['z']  # Left and right targets: trials x epochs x sides
            fs_p300 = p300_features['fs']
            print(f"Loaded p300 features: {X_p300.shape}")

        # Flatten z_p300 to (trials * epochs, sides)
        z_p300_flat = z_p300.reshape(-1, z_p300.shape[2])  # Shape: (trials * epochs, sides)

        # Find the number of epochs per trial
        epochs_per_trial = X_p300.shape[1]  # Number of epochs per trial

        # Initialize lists to store averaged epochs for each trial
        left_target_averaged_trials = []
        right_target_averaged_trials = []

        # Loop over each trial
        for i_trial in range(X_p300.shape[0]):
            # Extract the epochs for the current trial (shape: epochs x channels x features)
            trial_epochs = X_p300[i_trial]

            # Extract the target labels for the current trial (shape: epochs x 2)
            trial_targets = z_p300_flat[i_trial * epochs_per_trial: (i_trial + 1) * epochs_per_trial]

            # Average epochs for left target (assuming 1 = left target)
            left_target_epochs = trial_epochs[trial_targets[:, 0] == 1]
            if len(left_target_epochs) > 0:
                left_target_averaged = np.mean(left_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                left_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no left target, set to zeros

            # Average epochs for right target (assuming 1 = right target)
            right_target_epochs = trial_epochs[trial_targets[:, 1] == 1]
            if len(right_target_epochs) > 0:
                right_target_averaged = np.mean(right_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                right_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no right target, set to zeros

            # Store the averaged epochs for the current trial
            left_target_averaged_trials.append(left_target_averaged)
            right_target_averaged_trials.append(right_target_averaged)

        # Convert lists to numpy arrays
        left_target_averaged_trials = np.array(left_target_averaged_trials)
        right_target_averaged_trials = np.array(right_target_averaged_trials)

        # Concatenate the averaged features from both groups (left and right)
        X_p300_averaged = np.concatenate([left_target_averaged_trials, right_target_averaged_trials], axis=1)

        # Flatten the last two dimensions (channels and features)
        X_p300_averaged_flat = X_p300_averaged.reshape(X_p300_averaged.shape[0], -1)
        
        # Concatenate the features of the P300, alpha and c-VEP
        X_combined = np.concatenate([X_p300_averaged_flat, X_alpha, X_cvep], axis=1)

        # Cross-validation setup
        fold_accuracies = []
        fold_roc_auc = []
        n_folds = 4
        n_trials = X_combined.shape[0] // n_folds
        folds = np.repeat(np.arange(n_folds), n_trials)

        for i_fold in range(n_folds):
            print(f"  Fold {i_fold + 1}/{n_folds}")

            # Split train and test data
            X_trn, y_trn = X_combined[folds != i_fold, :], y_cvep[folds != i_fold]
            X_tst, y_tst = X_combined[folds == i_fold, :], y_cvep[folds == i_fold]

            # LDA classifier
            lda = LinearDiscriminantAnalysis(solver="lsqr", covariance_estimator=LedoitWolf())
            lda.fit(X_trn, y_trn)

            # Make predictions
            y_pred = lda.predict(X_tst)

            # Compute performance metrics
            accuracy = accuracy_score(y_tst, y_pred)
            print(f"Fold {i_fold+1} accuracy: {accuracy}")

            fold_accuracies.append(accuracy)

        # Calculate average accuracy over all folds
        average_accuracy = np.mean(fold_accuracies)
        subject_accuracies.append(fold_accuracies)
        subject_std.append(np.std(fold_accuracies))  # compute std over folds
        print(f"Average accuracy over all folds: {average_accuracy}\n STD: {np.std(fold_accuracies):.2f}")

grand_average_accuracy = np.mean(subject_accuracies)
print(f"\nGrand average accuracy across all subjects: {grand_average_accuracy:.4f}")

# Convert list to array
subject_accuracies_array = np.array(subject_accuracies)
subject_std_array = np.array(subject_std)

# Save the results
save_path = os.path.join(save_dir, 'p300_cvep_hybrid_accuracy_results.npz')
np.savez(save_path,
         accuracy=subject_accuracies_array,
         std=subject_std_array,
         subjects=subjects,
         tasks=tasks,
         n_folds=n_folds,
         method='hybrid')

print(f"\nSaved results to: {save_path}")

Loading features for VPpdia, task=covert
Loaded c-VEP features: (80, 2)
Loaded p300 features: (80, 80, 62, 6)
  Fold 1/4
Fold 1 accuracy: 0.65
  Fold 2/4
Fold 2 accuracy: 0.7
  Fold 3/4
Fold 3 accuracy: 0.7
  Fold 4/4
Fold 4 accuracy: 0.8
Average accuracy over all folds: 0.7124999999999999
 STD: 0.05
Loading features for VPpdib, task=covert
Loaded c-VEP features: (80, 2)
Loaded p300 features: (80, 80, 61, 6)
  Fold 1/4
Fold 1 accuracy: 0.75
  Fold 2/4
Fold 2 accuracy: 0.65
  Fold 3/4
Fold 3 accuracy: 0.85
  Fold 4/4
Fold 4 accuracy: 0.65
Average accuracy over all folds: 0.725
 STD: 0.08
Loading features for VPpdic, task=covert
Loaded c-VEP features: (80, 2)
Loaded p300 features: (80, 80, 63, 6)
  Fold 1/4
Fold 1 accuracy: 0.55
  Fold 2/4
Fold 2 accuracy: 0.65
  Fold 3/4
Fold 3 accuracy: 0.6
  Fold 4/4
Fold 4 accuracy: 0.75
Average accuracy over all folds: 0.6375000000000001
 STD: 0.07
Loading features for VPpdid, task=covert
Loaded c-VEP features: (80, 2)
Loaded p300 features: (80, 80,

Fold 1 accuracy: 0.95
  Fold 2/4
Fold 2 accuracy: 0.95
  Fold 3/4
Fold 3 accuracy: 0.95
  Fold 4/4
Fold 4 accuracy: 0.85
Average accuracy over all folds: 0.9249999999999999
 STD: 0.04

Grand average accuracy across all subjects: 0.8164

Saved results to: /Users/juliette/Desktop/thesis/results/hybrid_simple/p300+cvep/p300_cvep_hybrid_accuracy_results.npz


## Using BT-LDA
Starting with hybrid decoding, now using BT-LDA.

In [28]:
from toeplitzlda.classification import ToeplitzLDA

# Paths
cvep_features_dir = '/Users/juliette/Desktop/thesis/features/c-VEP'  # where c-VEP features are stored
alpha_features_dir = '/Users/juliette/Desktop/thesis/features/alpha'  # where alpha features are stored
p300_features_dir = '/Users/juliette/Desktop/thesis/preprocessing/features/with_ICA'  # where P300 features are stored
save_dir = '/Users/juliette/Desktop/thesis/results/hybrid_simple/alpha+p300+c-vep'

# Subject and task lists
subjects = [
    "VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij", "VPpdik",
    "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit", "VPpdiu", "VPpdiv",
    "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"
]
tasks = ["covert"]
subject_accuracies = []
subject_std = []

# Loop over subjects and tasks
for subject in subjects:
    for task in tasks:
        print(f"Loading features for {subject}, task={task}")

        # Load c-VEP features
        cvep_file_path = os.path.join(cvep_features_dir, f"sub-{subject}_task-{task}_features.npz")
        if os.path.exists(cvep_file_path):
            cvep_data = np.load(cvep_file_path)
            X_cvep = cvep_data['features']  # Features for c-VEP task (trials x features)
            y_cvep = cvep_data['y']  # Labels for c-VEP task
            print(f"Loaded c-VEP features: {X_cvep.shape}")
        else:
            print(f"Warning: c-VEP features file not found for {subject}, task={task}")

        # Load alpha features
        alpha_file_path = os.path.join(alpha_features_dir, f"sub-{subject}_task-{task}_alpha_features.npz")
        if os.path.exists(alpha_file_path):
            alpha_data = np.load(alpha_file_path)
            X_alpha = alpha_data['features']  # Features for alpha task (trials x features)
            y_alpha = alpha_data['labels']  # Labels for alpha task
            print(f"Loaded alpha features: {X_alpha.shape}")
        else:
            print(f"Warning: alpha features file not found for {subject}, task={task}")

        # Load P300 features
        file_path_p300 = os.path.join(p300_features_dir, f"sub-{subject}", f"sub-{subject}_task-{task}_p300_features_ICA.npz")
        if os.path.exists(file_path_p300):
            p300_features = np.load(file_path_p300)

            X_p300 = p300_features['X']  # Shape: trials x epochs x channels x features
            y_p300 = p300_features['y']  # Labels indicating cued side: trials
            z_p300 = p300_features['z']  # Left and right targets: trials x epochs x sides
            fs_p300 = p300_features['fs']
            print(f"Loaded p300 features: {X_p300.shape}")
        n_channels = X_p300[2]
        # Flatten z_p300 to (trials * epochs, sides)
        z_p300_flat = z_p300.reshape(-1, z_p300.shape[2])  # Shape: (trials * epochs, sides)

        # Find the number of epochs per trial
        epochs_per_trial = X_p300.shape[1]  # Number of epochs per trial

        # Initialize lists to store averaged epochs for each trial
        left_target_averaged_trials = []
        right_target_averaged_trials = []

        # Loop over each trial
        for i_trial in range(X_p300.shape[0]):
            # Extract the epochs for the current trial (shape: epochs x channels x features)
            trial_epochs = X_p300[i_trial]

            # Extract the target labels for the current trial (shape: epochs x 2)
            trial_targets = z_p300_flat[i_trial * epochs_per_trial: (i_trial + 1) * epochs_per_trial]

            # Average epochs for left target (assuming 1 = left target)
            left_target_epochs = trial_epochs[trial_targets[:, 0] == 1]
            if len(left_target_epochs) > 0:
                left_target_averaged = np.mean(left_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                left_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no left target, set to zeros

            # Average epochs for right target (assuming 1 = right target)
            right_target_epochs = trial_epochs[trial_targets[:, 1] == 1]
            if len(right_target_epochs) > 0:
                right_target_averaged = np.mean(right_target_epochs, axis=0)  # Average across epochs (axis=0)
            else:
                right_target_averaged = np.zeros(trial_epochs.shape[1:])  # If no right target, set to zeros

            # Store the averaged epochs for the current trial
            left_target_averaged_trials.append(left_target_averaged)
            right_target_averaged_trials.append(right_target_averaged)

        # Convert lists to numpy arrays
        left_target_averaged_trials = np.array(left_target_averaged_trials)
        right_target_averaged_trials = np.array(right_target_averaged_trials)

        # Concatenate the averaged features from both groups (left and right)
        X_p300_averaged = np.concatenate([left_target_averaged_trials, right_target_averaged_trials], axis=1)

        # Flatten the last two dimensions (channels and features)
        X_p300_averaged_flat = X_p300_averaged.reshape(X_p300_averaged.shape[0], -1)
        print("SHAPE of X_p300_averaged_flat:", X_p300_averaged_flat.shape)
        # Concatenate the features of the P300, alpha and c-VEP
        X_combined = np.concatenate([X_p300_averaged_flat, X_alpha, X_cvep], axis=1)

        # Cross-validation setup
        fold_accuracies = []
        fold_roc_auc = []
        n_folds = 4
        n_trials = X_combined.shape[0] // n_folds
        folds = np.repeat(np.arange(n_folds), n_trials)

        for i_fold in range(n_folds):
            print(f"  Fold {i_fold + 1}/{n_folds}")

            # Split train and test data
            X_trn, y_trn = X_combined[folds != i_fold, :], y_cvep[folds != i_fold]
            X_tst, y_tst = X_combined[folds == i_fold, :], y_cvep[folds == i_fold]
            
            # LDA classifier
            ## Fit LDA
            print("SHAPE OF X_tst:", X_tst.shape)
            toeplitz = ToeplitzLDA(n_channels=n_channels)
            # Dimensionality bust be 2
            toeplitz.fit(X_trn, y_trn)

            # Make predictions
            y_pred = lda.predict(X_tst)

            # Compute performance metrics
            accuracy = accuracy_score(y_tst, y_pred)
            print(f"Fold {i_fold+1} accuracy: {accuracy}")

            fold_accuracies.append(accuracy)

        # Calculate average accuracy over all folds
        average_accuracy = np.mean(fold_accuracies)
        subject_accuracies.append(fold_accuracies)
        subject_std.append(np.std(fold_accuracies))  # compute std over folds
        print(f"Average accuracy over all folds: {average_accuracy}\n STD: {np.std(fold_accuracies):.2f}")

grand_average_accuracy = np.mean(subject_accuracies)
print(f"\nGrand average accuracy across all subjects: {grand_average_accuracy:.4f}")

# Convert list to array
subject_accuracies_array = np.array(subject_accuracies)
subject_std_array = np.array(subject_std)

# # Save the results
# save_path = os.path.join(save_dir, 'cvep_p300_alpha_hybrid_accuracy_results.npz')
# np.savez(save_path,
#          accuracy=subject_accuracies_array,
#          std=subject_std_array,
#          subjects=subjects,
#          tasks=tasks,
#          n_folds=n_folds,
#          method='hybrid')

print(f"\nSaved results to: {save_path}")

Loading features for VPpdia, task=covert
Loaded c-VEP features: (80, 2)
Loaded alpha features: (80, 4)
Loaded p300 features: (80, 80, 62, 6)
SHAPE of X_p300_averaged_flat: (80, 744)
  Fold 1/4
SHAPE OF X_tst: (20, 750)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()